In [1]:
#import different algorithms, models n libraries

import numpy as np; import pandas as pd
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#We are trying to find the best possible model. 
#For any model to be the best, we should be able 
#to supply best possible parameters.
#If we manually try to find out the best params,
#it will be too tedious and time consuming.
#Hence we will use GridSearchCV model and avoid
#all other models that we knew already.
#I have worked out all known manual method below.
#Finally, GridSearchCV model is used at the end.

In [2]:

digits = load_digits()
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

In [3]:
digits.data[:1]

array([[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
        15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
        12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
         0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
        10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.]])

In [4]:
digits.feature_names[:5]

['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4']

In [5]:
len(digits.feature_names)

64

In [6]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

In [7]:
len(digits.target)

1797

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
#Approach#01: train_test_split to manually tune params (trial n error)
X_train,X_test,y_train,y_test = train_test_split(digits.data,digits.target,test_size=0.3)
model = SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.38333333333333336

In [9]:
#Approach#02: K-fold Cross Validation, mnually supply diff params
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),digits.data,digits.target,cv=5)


array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [10]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),digits.data,digits.target,cv=5)


array([0.45277778, 0.46944444, 0.47910864, 0.47910864, 0.50139276])

In [11]:
cross_val_score(SVC(kernel='linear',C=20,gamma='auto'),digits.data,digits.target,cv=5)


array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [12]:
#All above methods are tiresome and very manual.Now, use loop as an alternative
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_scores = {'kernel': [], 'C': [], 'score': []}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel=kval, C=cval, gamma='auto'), digits.data, digits.target, cv=5)
        avg_scores['kernel'].append(kval)
        avg_scores['C'].append(cval)
        avg_scores['score'].append(round(np.average(cv_scores), 3))
pd.DataFrame(avg_scores)

,kernel,C,score
0,rbf,1,0.449
1,rbf,10,0.476
2,rbf,20,0.476
3,linear,1,0.948
4,linear,10,0.948
5,linear,20,0.948


In [13]:
#Approac#3: GridSearchCV
#GridSearchCV does exactly as above but in a one-line code
#GridSearchCV being used in SVC below.
clf = GridSearchCV(SVC(gamma='auto'),{
    'C': [1,10,20],
    'kernel': ['rbf','linear']
    }, 
    cv=5, return_train_score=False)

clf.fit(digits.data,digits.target)
clf.cv_results_

{'mean_fit_time': array([0.31503501, 0.02501297, 0.30834508, 0.0252068 , 0.3023674 ,
        0.02495112]),
 'std_fit_time': array([0.00791813, 0.00115099, 0.00750845, 0.00111122, 0.00187156,
        0.00094643]),
 'mean_score_time': array([0.1209702 , 0.00674601, 0.13973184, 0.00701623, 0.12270689,
        0.00687356]),
 'std_score_time': array([0.00276299, 0.00015798, 0.04257886, 0.00029721, 0.00430212,
        0.00020345]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [14]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.315035,0.007918,0.120970,0.002763,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.411111,0.450000,0.454039,0.448468,0.479109,0.448545,0.021761,6
1,0.025013,0.001151,0.006746,0.000158,1,linear,"{'C': 1, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
2,0.308345,0.007508,0.139732,0.042579,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
3,0.025207,0.001111,0.007016,0.000297,10,linear,"{'C': 10, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
4,0.302367,0.001872,0.122707,0.004302,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
5,0.024951,0.000946,0.006874,0.000203,20,linear,"{'C': 20, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1


In [15]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.448545
1,1,linear,0.947697
2,10,rbf,0.476366
3,10,linear,0.947697
4,20,rbf,0.476366
5,20,linear,0.947697


In [16]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [17]:
clf.best_score_

0.9476973073351903

In [18]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [19]:
#The above lengthy code can be contracted
#to a very simple and short code by using 
#GridSearchCV model that will all the above
#used algorithms together and generate
#result to choose the best.

model_params = {
    #1
    'svm': {
        'model': SVC(gamma='auto') , 
        'params': {'C': [1,10,20], 'kernel': ['rbf','linear']}
    }, 
    #2
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {'n_estimators': [1,5,10]}
    }, 
    #3
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'), 
        'params': {'C': [1,5,10]}
    },
    #4
    'naive_bayes_gaussian': {
        'model': GaussianNB(), 
        'params': {}
    },
    #5
    'naive_bayes_multinomial': {
        'model': MultinomialNB(), 
        'params': {}
    },
    #6
    'decision_tree':{
        'model': DecisionTreeClassifier(), 
        'params': {'criterion': ['gini', 'entropy']}
    }
}

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(digits.data, digits.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
pd.DataFrame(scores)

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.908193,{'n_estimators': 10}
2,logistic_regression,0.922114,{'C': 1}
3,naive_bayes_gaussian,0.806928,{}
4,naive_bayes_multinomial,0.870350,{}
5,decision_tree,0.806360,{'criterion': 'entropy'}
